In [50]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data97586


In [51]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [52]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 122kB 18.4MB/s eta 0:00:01


In [53]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

In [54]:
# ## baseline1版本，不参与建模的特征 ['os', 'lan', 'sid’]
# ## Score = 88.2733
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [55]:
# 数据加载
train = pd.read_csv('/home/aistudio/data/data97586/train.csv')
test = pd.read_csv('/home/aistudio/data/data97586/test1.csv')
train

,Unnamed: 0,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,label,lan,media_id,...,os,osv,package,sid,timestamp,version,fea_hash,location,fea1_hash,cus_type
0,0,316361,1199,46000.0,0.0,0.0,0.0,1,NaN,104,...,android,9,18,1438873,1.559893e+12,8,2135019403,0,2329670524,601
1,1,135939,893,0.0,0.0,0.0,0.0,1,NaN,19,...,android,8.1,0,1185582,1.559994e+12,4,2782306428,1,2864801071,1000
2,2,399254,821,0.0,760.0,0.0,360.0,1,NaN,559,...,android,8.1.0,0,1555716,1.559837e+12,0,1392806005,2,628911675,696
3,3,68983,1004,46000.0,2214.0,0.0,1080.0,0,NaN,129,...,android,8.1.0,0,1093419,1.560042e+12,0,3562553457,3,1283809327,753
4,4,288999,1076,46000.0,2280.0,0.0,1080.0,1,zh-CN,64,...,android,8.0.0,0,1400089,1.559867e+12,5,2364522023,4,1510695983,582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,499995,392477,1028,46000.0,1920.0,3.0,1080.0,1,zh-CN,144,...,Android,7.1.2,25,1546078,1.559834e+12,7,861755946,79,140647032,373
499996,499996,346134,1001,0.0,1424.0,0.0,720.0,0,NaN,29,...,android,8.1.0,0,1480612,1.559814e+12,3,1714444511,23,2745131047,525
499997,499997,499635,761,46000.0,1280.0,0.0,720.0,0,NaN,54,...,android,6.0.1,9,1698442,1.559676e+12,0,3843262581,25,1326115882,810
499998,499998,239786,917,46001.0,960.0,0.0,540.0,0,zh_CN,109,...,android,5.1.1,0,1331155,1.559840e+12,0,1984296118,225,1446741112,772


In [56]:
# 字符串类型 需要转换为数值（labelencoder）
object_cols = train.select_dtypes(include='object').columns

In [57]:
# 缺失值个数
temp = train.isnull().sum()
# 有缺失值的字段： lan, osv
temp[temp>0]

lan    183280
osv      6561
dtype: int64

In [58]:
# ['os', 'osv', 'lan', 'sid’]
features = train.columns.tolist()
features.remove('label')
print(features)

['Unnamed: 0', 'android_id', 'apptype', 'carrier', 'dev_height', 'dev_ppi', 'dev_width', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package', 'sid', 'timestamp', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type']


In [59]:
for feature in features:
    print(feature, train[feature].nunique())

Unnamed: 0 500000
android_id 362258
apptype 89
carrier 5
dev_height 798
dev_ppi 92
dev_width 346
lan 21
media_id 284
ntt 8
os 2
osv 154
package 1950
sid 500000
timestamp 500000
version 22
fea_hash 402980
location 332
fea1_hash 4959
cus_type 58


In [60]:
# fea_hash是否要做特征变换
#train['fea_hash'].value_counts()
#train['fea_hash'].describe()
train['fea_hash'].map(lambda x: len(str(x))).value_counts()
#train['fea1_hash'].value_counts()
train['fea1_hash'].map(lambda x: len(str(x))).value_counts()

10    391669
9      99347
8       8977
7          6
5          1
Name: fea1_hash, dtype: int64

In [61]:
# 对osv进行数据清洗
def osv_trans(x):
    x = str(x).replace('Android_', '').replace('Android ', '').replace('W', '')
    if str(x).find('.')>0:
        temp_index1 = x.find('.')
        if x.find(' ')>0:
            temp_index2 = x.find(' ')
        else:
            temp_index2 = len(x)

        if x.find('-')>0:
            temp_index2 = x.find('-')
            
        result = x[0:temp_index1] + '.' + x[temp_index1+1:temp_index2].replace('.', '')
        try:
            return float(result)
        except:
            print(x+'#########')
            return 0
    try:
        return float(x)
    except:
        print(x+'#########')
        return 0

In [62]:
# 采用众数，进行缺失值的填充
train['osv'].fillna('8.1.0', inplace=True)
# 数据清洗
train['osv'] = train['osv'].apply(osv_trans)

f073b_changxiang_v01_b1b8_20180915#########
%E6%B1%9F%E7%81%B5OS+5.0#########


In [63]:
# 采用众数，进行缺失值的填充
test['osv'].fillna('8.1.0', inplace=True)
# 数据清洗
test['osv'] = test['osv'].apply(osv_trans)


GIONEE_YNGA#########


In [64]:
#train['os'].value_counts()
train['lan'].value_counts()
#lan_map = {'zh-CN': 1, }
train['lan'].value_counts().index
lan_map = {'zh-CN': 1, 'zh_CN':2, 'Zh-CN': 3, 'zh-cn': 4, 'zh_CN_#Hans':5, 
           'zh': 6, 'ZH': 7, 'cn':8, 'CN':9, 'zh-HK': 10, 'tw': 11, 
           'TW': 12,'zh-TW': 13,'zh-MO':14, 'en':15, 'en-GB': 16, 
           'en-US': 17, 'ko': 18, 'ja': 19, 'it': 20, 'mi':21} 
train['lan'] = train['lan'].map(lan_map)
test['lan'] = test['lan'].map(lan_map)
test['lan'].value_counts()


1.0     73046
6.0      9094
8.0      6136
2.0      3391
3.0      2430
4.0       165
7.0        60
9.0        46
11.0       28
15.0       26
5.0        11
18.0        6
13.0        5
17.0        2
10.0        2
12.0        1
16.0        1
Name: lan, dtype: int64

In [65]:
# 对于有缺失的lan 设置为22
train['lan'].fillna(22, inplace=True)
test['lan'].fillna(22, inplace=True)


In [66]:
remove_list = ['os',  'sid']
col = features
for i in remove_list:
    col.remove(i)
col

['Unnamed: 0',
 'android_id',
 'apptype',
 'carrier',
 'dev_height',
 'dev_ppi',
 'dev_width',
 'lan',
 'media_id',
 'ntt',
 'osv',
 'package',
 'timestamp',
 'version',
 'fea_hash',
 'location',
 'fea1_hash',
 'cus_type']

In [67]:

def version_trans(x):
    if x=='V3':
        return 3
    if x=='v1':
        return 1
    if x=='P_Final_6':
        return 6
    if x=='V6':
        return 6
    if x=='GA3':
        return 3
    if x=='GA2':
        return 2
    if x=='V2':
        return 2
    if x=='50':
        return 5
    return int(x)
train['version'] = train['version'].apply(version_trans)
test['version'] = test['version'].apply(version_trans)
train['version'] = train['version'].astype('int')
test['version'] = test['version'].astype('int')


In [68]:
#train['timestamp'].value_counts()
#train['timestamp'] = pd.to_datetime(train['timestamp'])
#train['timestamp']
from datetime import datetime

# lambda 是一句话函数，匿名函数
train['timestamp'] = train['timestamp'].apply(lambda x: datetime.fromtimestamp(x/1000))
#1559892728241.7212
#1559871800477.1477
#1625493942.538375
#import time
#time.time()
test['timestamp'] = test['timestamp'].apply(lambda x: datetime.fromtimestamp(x/1000))
test['timestamp']


0        2019-06-07 09:43:20.477148
1        2019-06-05 20:53:33.449314
2        2019-06-04 10:08:24.732394
3        2019-06-05 01:03:10.643892
4        2019-06-05 08:16:34.450110
                    ...            
149995   2019-06-08 09:21:33.629823
149996   2019-06-07 07:15:57.431662
149997   2019-06-09 08:48:59.661749
149998   2019-06-05 06:38:14.527250
149999   2019-06-08 07:23:16.149025
Name: timestamp, Length: 150000, dtype: datetime64[ns]

In [69]:
train['timestamp'] 

0        2019-06-07 15:32:08.241721
1        2019-06-08 19:40:20.922229
2        2019-06-06 23:58:17.549109
3        2019-06-09 08:59:37.719685
4        2019-06-07 08:28:21.478553
                    ...            
499995   2019-06-06 23:14:18.267242
499996   2019-06-06 17:40:54.045280
499997   2019-06-05 03:14:57.290064
499998   2019-06-07 00:59:27.651521
499999   2019-06-07 23:32:46.472229
Name: timestamp, Length: 500000, dtype: datetime64[ns]

In [70]:
# 特征筛选
features = train[col]
# 构造fea_hash_len特征
features['fea_hash_len'] = features['fea_hash'].map(lambda x: len(str(x)))
features['fea1_hash_len'] = features['fea1_hash'].map(lambda x: len(str(x)))
# Thinking：为什么将很大的，很长的fea_hash化为0？
# 如果fea_hash很长，都归为0，否则为自己的本身
features['fea_hash'] = features['fea_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
features['fea1_hash'] = features['fea1_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
features

,Unnamed: 0,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,lan,media_id,ntt,osv,package,timestamp,version,fea_hash,location,fea1_hash,cus_type,fea_hash_len,fea1_hash_len
0,0,316361,1199,46000.0,0.0,0.0,0.0,22.0,104,6.0,9.00,18,2019-06-07 15:32:08.241721,8,2135019403,0,2329670524,601,10,10
1,1,135939,893,0.0,0.0,0.0,0.0,22.0,19,6.0,8.10,0,2019-06-08 19:40:20.922229,4,2782306428,1,2864801071,1000,10,10
2,2,399254,821,0.0,760.0,0.0,360.0,22.0,559,0.0,8.10,0,2019-06-06 23:58:17.549109,0,1392806005,2,628911675,696,10,9
3,3,68983,1004,46000.0,2214.0,0.0,1080.0,22.0,129,2.0,8.10,0,2019-06-09 08:59:37.719685,0,3562553457,3,1283809327,753,10,10
4,4,288999,1076,46000.0,2280.0,0.0,1080.0,1.0,64,2.0,8.00,0,2019-06-07 08:28:21.478553,5,2364522023,4,1510695983,582,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,499995,392477,1028,46000.0,1920.0,3.0,1080.0,1.0,144,6.0,7.12,25,2019-06-06 23:14:18.267242,7,861755946,79,140647032,373,9,9
499996,499996,346134,1001,0.0,1424.0,0.0,720.0,22.0,29,2.0,8.10,0,2019-06-06 17:40:54.045280,3,1714444511,23,2745131047,525,10,10
499997,499997,499635,761,46000.0,1280.0,0.0,720.0,22.0,54,6.0,6.01,9,2019-06-05 03:14:57.290064,0,3843262581,25,1326115882,810,10,10
499998,499998,239786,917,46001.0,960.0,0.0,540.0,2.0,109,2.0,5.11,0,2019-06-07 00:59:27.651521,0,1984296118,225,1446741112,772,10,10


In [71]:
test_features = test[col]
# 构造fea_hash_len特征
test_features['fea_hash_len'] = test_features['fea_hash'].map(lambda x: len(str(x)))
test_features['fea1_hash_len'] = test_features['fea1_hash'].map(lambda x: len(str(x)))
# Thinking：为什么将很大的，很长的fea_hash化为0？
# 如果fea_hash很长，都归为0，否则为自己的本身
test_features['fea_hash'] = test_features['fea_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
test_features['fea1_hash'] = test_features['fea1_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
test_features

,Unnamed: 0,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,lan,media_id,ntt,osv,package,timestamp,version,fea_hash,location,fea1_hash,cus_type,fea_hash_len,fea1_hash_len
0,0,317625,1181,46000.0,2196.0,2.0,1080.0,9.0,639,2.0,8.10,188,2019-06-07 09:43:20.477148,7,1672223856,57,3872258917,658,10,10
1,1,435108,944,46003.0,2280.0,3.0,1080.0,1.0,704,6.0,8.10,221,2019-06-05 20:53:33.449314,3,3767901757,23,129322164,943,10,9
2,2,0,1106,46000.0,0.0,0.0,0.0,22.0,39,2.0,5.10,1562,2019-06-04 10:08:24.732394,0,454638703,30,4226678391,411,9,10
3,3,451504,761,46000.0,1344.0,0.0,720.0,22.0,54,2.0,7.11,9,2019-06-05 01:03:10.643892,0,1507622951,65,3355419572,848,10,10
4,4,0,1001,46000.0,665.0,0.0,320.0,1.0,29,5.0,8.10,4,2019-06-05 08:16:34.450110,0,4116351093,148,2644467751,411,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,149995,0,1001,46000.0,760.0,0.0,360.0,1.0,29,2.0,8.10,4,2019-06-08 09:21:33.629823,0,3162887451,126,2711576615,411,10,10
149996,149996,0,1001,46000.0,780.0,0.0,360.0,1.0,29,2.0,9.00,4,2019-06-07 07:15:57.431662,0,97238959,322,2678022183,411,8,10
149997,149997,0,1001,46000.0,780.0,0.0,360.0,1.0,29,5.0,8.10,4,2019-06-09 08:48:59.661749,0,1320118495,46,2610913319,411,10,10
149998,149998,500925,1052,46000.0,854.0,240.0,480.0,1.0,249,6.0,4.42,0,2019-06-05 06:38:14.527250,2,1292986591,41,1898209327,430,10,10


In [72]:
# 对训练集的timestamp提取时间多尺度

# 使用to_datetime进行日期类型转换
# 创建时间戳索引
temp = pd.DatetimeIndex(features['timestamp'])
features['year'] = temp.year
features['month'] = temp.month
features['day'] = temp.day
features['week_day'] = temp.weekday #星期几
features['hour'] = temp.hour
features['minute'] = temp.minute

In [73]:
# 求时间的diff
start_time = features['timestamp'].min()
features['time_diff'] = features['timestamp'] - start_time
features['time_diff'] = features['time_diff'].dt.days + features['time_diff'].dt.seconds/3600/24
features[['timestamp', 'year', 'month', 'day', 'week_day', 'hour', 'minute', 'time_diff']]


,timestamp,year,month,day,week_day,hour,minute,time_diff
0,2019-06-07 15:32:08.241721,2019,6,7,4,15,32,4.647303
1,2019-06-08 19:40:20.922229,2019,6,8,5,19,40,5.819664
2,2019-06-06 23:58:17.549109,2019,6,6,3,23,58,3.998796
3,2019-06-09 08:59:37.719685,2019,6,9,6,8,59,6.374722
4,2019-06-07 08:28:21.478553,2019,6,7,4,8,28,4.353009
...,...,...,...,...,...,...,...,...
499995,2019-06-06 23:14:18.267242,2019,6,6,3,23,14,3.968252
499996,2019-06-06 17:40:54.045280,2019,6,6,3,17,40,3.736713
499997,2019-06-05 03:14:57.290064,2019,6,5,2,3,14,2.135370
499998,2019-06-07 00:59:27.651521,2019,6,7,4,0,59,4.041273


In [74]:
# 创建时间戳索引
temp = pd.DatetimeIndex(test_features['timestamp'])
test_features['year'] = temp.year
test_features['month'] = temp.month
test_features['day'] = temp.day
test_features['week_day'] = temp.weekday #星期几
test_features['hour'] = temp.hour
test_features['minute'] = temp.minute


In [75]:
# 求时间的diff
#start_time = features['timestamp'].min()
test_features['time_diff'] = test_features['timestamp'] - start_time
test_features['time_diff'] = test_features['time_diff'].dt.days + test_features['time_diff'].dt.seconds/3600/24
#test_features[['timestamp', 'year', 'month', 'day', 'week_day', 'hour', 'minute', 'time_diff']]
test_features['time_diff']

0         4.405081
1         2.870509
2         1.422488
3         2.043854
4         2.344826
            ...   
149995    5.389954
149996    4.302731
149997    6.367338
149998    2.276539
149999    5.307801
Name: time_diff, Length: 150000, dtype: float64

In [81]:
import re
features = features.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [82]:
test_features = test_features.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [83]:
#train['os'].value_counts()
# 使用LGBM训练
import lightgbm as lgb
model = lgb.LGBMClassifier()
# 模型训练
model.fit(features.drop(['timestamp'], axis=1), train['label'])
result = model.predict(test_features.drop(['timestamp'], axis=1))
result

array([0, 1, 0, ..., 1, 1, 1])

In [84]:
#features['version'].value_counts()
res = pd.DataFrame(test['sid'])
res['label'] = result
res.to_csv('./baseline_lan_timpstamp.csv', index=False)
res

,sid,label
0,1440682,0
1,1606824,1
2,1774642,0
3,1742535,0
4,1689686,1
...,...,...
149995,1165373,1
149996,1444115,1
149997,1134378,1
149998,1700238,1
